In [1]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.5 MB/s eta 0:00:00


In [2]:
## Embedding
!pip install install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 4.1 MB/s eta 0:00:00


In [3]:
!pip install llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [4]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader,ServiceContext, download_loader
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts.prompts import SimpleInputPrompt

In [5]:

DATA_URL = ["https://www.acko.com/life-insurance/", "https://www.acko.com/health-insurance/for-parents/", "https://www.acko.com/car-insurance/","https://www.acko.com/two-wheeler-insurance/"]

BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")

loader = BeautifulSoupWebReader()
documents = loader.load_data(urls=DATA_URL)

In [6]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [7]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [8]:
import torch
llm = HuggingFaceLLM( context_window=4096,
                      max_new_tokens=256,
                      generate_kwargs={"temperature": 0.0, "do_sample": False},
                      system_prompt=system_prompt,
                      query_wrapper_prompt=query_wrapper_prompt,
                      tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
                      model_name="meta-llama/Llama-2-7b-chat-hf",
                      device_map="auto",
                      # uncomment this if using CUDA to reduce memory usage
                      model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True} )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [9]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import ServiceContext
from llama_index.embeddings import LangchainEmbedding
embed_model=LangchainEmbedding( HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
service_context=ServiceContext.from_defaults( chunk_size=1024, llm=llm, embed_model=embed_model )

In [11]:
index=VectorStoreIndex.from_documents(documents,service_context=service_context)

In [12]:
query_engine=index.as_query_engine()

In [13]:
response=query_engine.query("what are the benefits of car insurance from acko?")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(



Based on the context information provided, the benefits of car insurance from ACKO are:

1. Stress-free claims experience with features such as free pickup and drop, and claim tracking.
2. Instant claim settlement for minor claims.
3. Total convenience for buying car insurance online anytime, anywhere.
4. 24x7 support.
5. Different types of car insurance policies to choose from, including third-party car insurance, comprehensive car insurance, zero depreciation policy, standalone own damage cover policy, and third-party v/s comprehensive car insurance policy.
6. Add-on covers available for enhanced policy coverage.
7. Legally mandatory third-party car insurance policy.
8. Coverage for damages to your car caused by accidents, natural disasters, fire, theft, and rat bites.

Please let me know if you have any further questions or if there's anything else I can help you with.


In [14]:
response=query_engine.query("what is the process of buying a health insurance?")
print("\n\n\n----------------Response-----------------\n\n\n")
print(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(





----------------Response-----------------



The process of buying a health insurance policy for your parents involves several steps:

1. Research and compare different insurance providers and policies to find the one that best suits your parents' needs and budget.
2. Check the policy documents and terms and conditions to understand what is covered and what is not.
3. Share the necessary details and medical history of your parents with the insurance provider to get a quote.
4. Pay the premium online or through other payment methods accepted by the insurance provider.
5. Wait for the medical check-up to be conducted by the insurance provider to assess your parents' health status.
6. Once the policy is issued, you can start enjoying the coverage provided by the policy.

Note: The process may vary depending on the insurance provider and the specific policy you choose.


In [15]:
response=query_engine.query("compare the car insurance benefit and bike insurance benefit?")
print("\n\n\n----------------Response-----------------\n\n\n")
print(response)




----------------Response-----------------




Bike insurance and car insurance offer different benefits. Bike insurance typically covers damages to the two-wheeler due to accidents, natural disasters, theft, and third-party liabilities. On the other hand, car insurance covers damages to the vehicle due to accidents, theft, and third-party liabilities.

In terms of benefits, bike insurance typically offers lower premiums compared to car insurance. This is because bikes are generally less expensive than cars and are less likely to be involved in accidents. Additionally, bike insurance policies often have a lower deductible, which means that the policyholder has to pay less out-of-pocket expenses in the event of a claim.

Car insurance, on the other hand, typically offers more comprehensive coverage, including damage to other vehicles or property in an accident, as well as personal injury protection. Car insurance policies also often have higher premiums than bike insurance policies, r

In [16]:
response=query_engine.query("suggest best health insurance for 72 year old person?")
print("\n\n\n----------------Response-----------------\n\n\n")
print(response)




----------------Response-----------------




Based on the provided context information, I would suggest the following health insurance plan for a 72-year-old person:

1. ACKO General Insurance's Senior Citizen Health Insurance Plan: This plan is specifically designed for individuals above the age of 60 years, providing coverage for medical expenses, including pre-existing conditions, hospitalization, and daycare treatments. The plan also offers a lifelong renewability feature and a fixed sum insured, which can be used to cover medical expenses.

Key Features:

* Coverage for medical expenses, including pre-existing conditions
* Hospitalization and daycare treatments covered
* Lifelong renewability feature
* Fixed sum insured
* Cashless treatment at network hospitals

Why this plan?

* The plan is designed specifically for senior citizens, providing coverage for age-related medical conditions.
* The fixed sum insured can be used to cover medical expenses, including pre-existing cond

In [17]:
response=query_engine.query("what is difference between life insurance and health insurance?")
print("\n\n\n----------------Response-----------------\n\n\n")
print(response)




----------------Response-----------------



Life insurance and health insurance are two distinct types of insurance policies. While life insurance provides financial protection to your loved ones in the event of your untimely death, health insurance covers medical expenses incurred due to illnesses or injuries. In other words, life insurance focuses on providing a financial safety net after your demise, while health insurance focuses on protecting you during your lifetime.

Life insurance policies typically offer a lump-sum payout to your beneficiaries upon your death, while health insurance policies reimburse you for medical expenses incurred due to illnesses or injuries. The premiums for life insurance are generally higher than those for health insurance, as life insurance policies provide a broader level of financial protection.

In summary, life insurance is designed to provide financial security for your loved ones after your death, while health insurance is designed to protec

In [18]:
response=query_engine.query("Different Types of Car Insurance Policies offered by acko?")
print("\n\n\n----------------Response-----------------\n\n\n")
print(response)




----------------Response-----------------



ACKO offers different types of car insurance policies, including:

1. Third Party Car Insurance Policy: A mandatory policy that covers financial liabilities when you accidentally damage other people's property or cause injuries/death of a third party.
2. Comprehensive Car Insurance Policy: A policy that provides coverage for damages to your car caused by accidents, natural disasters, fire, or theft, in addition to third-party liabilities.
3. Zero Depreciation Policy - Super Saver (Bumper-to-Bumper): A policy that provides 100% coverage for the cost of car parts if a part needs to be replaced during repair.
4. Stand Alone Own Damage Cover Policy: A policy that provides coverage for damages to your car from accidents, natural disasters, fire, or theft, when a third-party liability policy is not enough.

Note that these policies, features, and add-ons are subject to availability and may vary based on the location and other factors. It is alw